This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5) and depends on Magma (tested using version
2.26-9).

In this notebook, we find generic curves of genus 6 which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1 with $\#C(\mathbb{F}_2) \in \{4,6\}$.  Allow 1.5 hours for completion.

In [1]:
load("preamble.sage")

In [2]:
F = GF(2)

In [3]:
P.<x01,x02,x03,x04,x12,x13,x14,x23,x24,x34> = PolynomialRing(F, 10)
quads = (x01*x23 + x02*x13 + x03*x12,
         x01*x24 + x02*x14 + x04*x12,
         x01*x34 + x03*x14 + x04*x13,
         x02*x34 + x03*x24 + x04*x23,
         x12*x34 + x13*x24 + x14*x23)

In [4]:
def vecs_to_gens(vecs):
    return tuple(sum(P.gens()[i] * v[i] for i in range(10)) for v in vecs)

In [5]:
with open("genus6-flats.txt", "r") as f:
    s = f.read()
    l = sage_eval(s)
    l = [[vector(F, v) for v in vecs] for vecs in l]
len(l)

57

In [6]:
monos2 = [prod(x) for x in itertools.combinations_with_replacement(P.gens(), 2)]
len(monos2)

55

In [7]:
coords = {}
for V in VectorSpace(F, 5).subspaces(2):
    M = V.matrix()
    M.set_immutable()
    coords[M] = vector(M.minors(2))
S = list(coords.keys())
len(S)

155

In [8]:
coords2 = {}
V0 = VectorSpace(F, 10)
for x in V0:
    x.set_immutable()
    coords2[x] = vector(F, (mu(*x) for mu in monos2))

In [9]:
def redundancy(gens, F=F, P=P, monos2=monos2, quads=quads):
    return [vector(F, ((gen*y).coefficient(mu) for mu in monos2)) for gen in gens for y in P.gens()] + \
       [vector(F, (gen.coefficient(mu) for mu in monos2)) for gen in quads]

In [10]:
curves = defaultdict(list)
for vecs in l:
    V1 = Matrix(vecs).row_space()
    V = Matrix(vecs).right_kernel()
    gens = vecs_to_gens(V1.basis())
    pts = [coords[x] for x in S if coords[x] in V]
    print(len(pts), [(s, len(curves[s])) for s in curves])
    for v in pts:
        v.set_immutable()
    W = VectorSpace(F, 55)
    W1 = W.quotient(redundancy(gens))
    perp = Matrix([coords2[x] for x in pts])
    for s in [4, 6]:
        for pts1 in itertools.combinations(pts, s):
            target = vector(F, (0 if x in pts1 else 1 for x in pts))
            for w in solve_right_iterator(perp, target, redundancy, gens):
                gens1 = sum(w[i]*monos2[i] for i in range(55))
                curves[(s,)].append(gens + (gens1,))

9 []
9 [((4,), 16128), ((6,), 10752)]
11 [((4,), 32256), ((6,), 21504)]
7 [((4,), 42816), ((6,), 36288)]
9 [((4,), 60736), ((6,), 39872)]
7 [((4,), 76864), ((6,), 50624)]
11 [((4,), 94784), ((6,), 54208)]
15 [((4,), 105344), ((6,), 68992)]
7 [((4,), 108032), ((6,), 79072)]
5 [((4,), 125952), ((6,), 82656)]
13 [((4,), 136192), ((6,), 82656)]
13 [((4,), 141912), ((6,), 96384)]
13 [((4,), 147632), ((6,), 110112)]
9 [((4,), 153232), ((6,), 124000)]
5 [((4,), 169360), ((6,), 134752)]
7 [((4,), 179600), ((6,), 134752)]
11 [((4,), 197520), ((6,), 138336)]
11 [((4,), 208272), ((6,), 152672)]
15 [((4,), 219024), ((6,), 167008)]
9 [((4,), 221712), ((6,), 177088)]
13 [((4,), 237840), ((6,), 187840)]
17 [((4,), 243560), ((6,), 201568)]
7 [((4,), 244750), ((6,), 207742)]
7 [((4,), 262670), ((6,), 211326)]
19 [((4,), 280590), ((6,), 214910)]
11 [((4,), 281086), ((6,), 218270)]
15 [((4,), 291646), ((6,), 233054)]
11 [((4,), 294376), ((6,), 243064)]
11 [((4,), 304936), ((6,), 257848)]
15 [((4,), 31549

In [11]:
F4 = GF(4)
S4 = []
for V in VectorSpace(F4, 5).subspaces(2):
    M = V.matrix()
    v = vector(M.minors(2))
    i = min(j for j in range(10) if v[j])
    assert v[i] == 1
    v.set_immutable()
    S4.append(v)
len(S4)

5797

In [12]:
tmp = [s for s in curves if len(s) == 1]
tmp2 = set(t[:2] for t in targets6)
for (s1,) in tmp:
    gens1 = 0
    for gens in curves[(s1,)]:
        if gens1 != gens[:-1]:
            gens1 = gens[:-1]
            pts2 = [x for x in S4 if all(gen(*x) == 0 for gen in gens1)]
        s2 = sum(1 for x in pts2 if gens[-1](*x) == 0)
        if (s1, s2) in tmp2:
            curves[(s1,s2)].append(gens)
    del curves[(s1,)]

In [13]:
[(s, len(curves[s])) for s in curves]

[((4, 14), 32577), ((4, 16), 21823), ((6, 14), 62493), ((6, 10), 86568)]

In [14]:
def count_by_ideal(gens, n):
    J = P.ideal(gens + quads + tuple(y^(2^n) + y for y in P.gens()))
    return (J.vector_space_dimension() - 1) // (2^n-1)

In [15]:
for n in range(3, 5):
    tmp = set(t[:n] for t in targets6)
    tmp2 = list(curves.keys())    
    for s in tmp2:
        for gens in curves[s]:
            i = count_by_ideal(gens, n)
            s1 = s + (i,)
            if s1 in tmp:
                curves[s1].append(gens)
        del curves[s]
    print([(s, len(curves[s])) for s in curves])

[((4, 14, 16), 8435), ((4, 16, 16), 6557), ((6, 14, 12), 3647), ((6, 14, 6), 1393), ((6, 10, 9), 4789)]
[((4, 14, 16, 18), 82), ((4, 14, 16, 26), 38), ((6, 14, 12, 26), 24), ((6, 14, 6, 34), 18), ((6, 14, 6, 26), 105), ((6, 10, 9, 38), 66)]


In [16]:
proj = magma.ProjectiveSpace(P)
curves = {s: [gens + quads for gens in curves[s]] for s in curves}
closeout(curves, X=proj, genus=6)

Number of curves found: 20
Number of isomorphism classes found: 7
Number of covers found: 1
(6, 14, 12, 26, 6, 56) ($.1^5 + $.1^3 + $.1)*$.1^4 + ($.1^6 + $.1^2 + $.1)*$.1^3 + ($.1^7 + $.1^2 + $.1)*$.1^2 + ($.1^8 + $.1^7 + $.1^6 + $.1^2 + 1)*$.1 + $.1^7 + $.1^6 + $.1^5 + $.1^2 + $.1 + 1
All covers recorded!
Total time: 86 minutes
